<a href="https://colab.research.google.com/github/umang66782/Plaksha/blob/main/A_B_testing_bayesian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Chance to beat all and Expected Potential Loss in an A/B test for a given number of visitors and number of conversions corresponding to both variations.

In [36]:
from scipy import stats
import numpy as np
 
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import matplotlib.gridspec as gridspec

In [37]:
data = {
    'variation_a': {
        'con': np.array([50,20,30,10,5], 
                      dtype=np.float),
        'num': np.array([100], 
                      dtype=np.float)
    },
    'variation_b': {
        'con': np.array([60,40,50,20,15], 
                      dtype=np.float),
        'num': np.array([100], 
                      dtype=np.float)
    }
}

In [39]:
#probability b better than a (prob_ba)
#probability b better than a error (prob_ba_error)
n_montocarlo = 100000
proba_b_a = []
proba_b_a_error = []
expected_loss_a = []
expected_loss_a_error = []
expected_loss_b = []
expected_loss_b_error = []
# mean and variance of a
mean_a, var_a = stats.beta.stats(a=1+sum(data['variation_a']['con'][:1]),b=1+(sum(data['variation_a']['num'][:1])-sum(data['variation_a']['con'][:1])),moments='mv')
# mean and variance of b
mean_b, var_b = stats.beta.stats(a=1+sum(data['variation_b']['con'][:1]),b=1+(sum(data['variation_b']['num'][:1])-sum(data['variation_b']['con'][:1])),moments='mv')
#genrating random values
rand_a = np.random.normal(loc=mean_a, scale=1.25*np.sqrt(var_a), size=n_montocarlo)
rand_b = np.random.normal(loc=mean_b, scale=1.25*np.sqrt(var_b), size=n_montocarlo)
#beta distribution of a and b
beta_a = stats.beta.pdf(rand_a, a=1+sum(data['variation_a']['con'][:1]), b=1+(sum(data['variation_a']['num'][:1])-sum(data['variation_a']['con'][:1])))
beta_b = stats.beta.pdf(rand_b, a=1+sum(data['variation_b']['con'][:1]) ,b=1+(sum(data['variation_b']['num'][:1])-sum(data['variation_b']['con'][:1])))
#normal distribution of a and b
norm_a = stats.norm.pdf(rand_a, loc=mean_a, scale=1.25*np.sqrt(var_a))
norm_b = stats.norm.pdf(rand_b, loc=mean_b, scale=1.25*np.sqrt(var_b))
z = (beta_a*beta_b) / (norm_a*norm_b)
# b is greater than a
z_b = z[rand_b>=rand_a]
prob = (1/n_montocarlo)* sum(z_b)
prob_err = np.sqrt(1*(z_b*z_b).sum()/n_montocarlo - (1*z_b.sum()/n_montocarlo)**2)/np.sqrt(n_montocarlo)
z_loss_a = ((rand_b-rand_a)*z)[rand_b>=rand_a]
loss_a = 1/n_montocarlo * sum(z_loss_a)
loss_a_err = np.sqrt(1*(z_loss_a*z_loss_a).sum()/n_montocarlo - (1*z_loss_a.sum()/n_montocarlo)**2)/np.sqrt(n_montocarlo)
z_loss_b = ((rand_a-rand_b)*z)[rand_a>=rand_b]
loss_b = 1/n_montocarlo * sum(z_loss_b)
loss_b_err = np.sqrt(1*(z_loss_b*z_loss_b).sum()/n_montocarlo - (1*z_loss_b.sum()/n_montocarlo)**2)/np.sqrt(n_montocarlo)
proba_b_a.append(prob)
proba_b_a_error.append(prob_err)
expected_loss_a.append(loss_a)
expected_loss_a_error.append(loss_a_err)
expected_loss_b.append(loss_b)
expected_loss_b_error.append(loss_b_err)

In [40]:
#probability of b conversion with respect of a
proba_b_a

[0.921054528055004]

In [41]:
proba_b_a_error

[0.0015545787549156428]

In [42]:
expected_loss_a

[0.10021704332613941]